*   Nama : Alfin Muhammad Ilmi
*   Kelas : M06
*   M299X0762

In [11]:
import pandas as pd

d1 = pd.read_csv('train.csv')
d2 = pd.read_csv('test.csv')
df = pd.concat([d1, d2])

df.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


Keterangan Class :
* 1-World 
* 2-Sports
* 3-Business
* 4-Sci/Tech

In [12]:
df['News'] = df['Title'] + ' ' + df['Description']
df.head()

,Class Index,Title,Description,News
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


In [13]:
df = df.drop(columns=['Title','Description'])
df

,Class Index,News
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."
...,...,...
7595,1,Around the world Ukrainian presidential candid...
7596,2,Void is filled with Clement With the supply of...
7597,2,Martinez leaves bitter Like Roger Clemens did ...
7598,3,5 of arthritis patients in Singapore take Bext...


In [14]:
category = pd.get_dummies(df['Class Index'])
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='Class Index')
df_baru

,News,1,2,3,4
0,Wall St. Bears Claw Back Into the Black (Reute...,0,0,1,0
1,Carlyle Looks Toward Commercial Aerospace (Reu...,0,0,1,0
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,0,0,1,0
3,Iraq Halts Oil Exports from Main Southern Pipe...,0,0,1,0
4,"Oil prices soar to all-time record, posing new...",0,0,1,0
...,...,...,...,...,...
7595,Around the world Ukrainian presidential candid...,1,0,0,0
7596,Void is filled with Clement With the supply of...,0,1,0,0
7597,Martinez leaves bitter Like Roger Clemens did ...,0,1,0,0
7598,5 of arthritis patients in Singapore take Bext...,0,0,1,0


In [15]:
news = df_baru['News'].values
label = df_baru[[1, 2, 3, 4]].values

In [16]:
from sklearn.model_selection import train_test_split
news_latih, news_val, label_latih, label_val = train_test_split(news, label, test_size=0.2)

# Data Cleaning

## Membuat lowercase

In [25]:
df['News'] = df['News'].apply(
    lambda data : data.lower()
)
df

,Class Index,News
0,3,wall st. bears claw back into the black (reute...
1,3,carlyle looks toward commercial aerospace (reu...
2,3,oil and economy cloud stocks' outlook (reuters...
3,3,iraq halts oil exports from main southern pipe...
4,3,"oil prices soar to all-time record, posing new..."
...,...,...
7595,1,around the world ukrainian presidential candid...
7596,2,void is filled with clement with the supply of...
7597,2,martinez leaves bitter like roger clemens did ...
7598,3,5 of arthritis patients in singapore take bext...


## Menghilangkan tanda baca

In [26]:
import string

def remove_tandabaca(text) : 
  nws = str.maketrans('', '', string.punctuation)
  return text.translate(nws)

In [28]:
df['News'] = df['News'].apply(
    lambda data : remove_tandabaca(data)
)
df

,Class Index,News
0,3,wall st bears claw back into the black reuters...
1,3,carlyle looks toward commercial aerospace reut...
2,3,oil and economy cloud stocks outlook reuters r...
3,3,iraq halts oil exports from main southern pipe...
4,3,oil prices soar to alltime record posing new m...
...,...,...
7595,1,around the world ukrainian presidential candid...
7596,2,void is filled with clement with the supply of...
7597,2,martinez leaves bitter like roger clemens did ...
7598,3,5 of arthritis patients in singapore take bext...


In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=8000, oov_token='<oov>')
tokenizer.fit_on_texts(news_latih)
 
sekuens_latih = tokenizer.texts_to_sequences(news_latih)
sekuens_val = tokenizer.texts_to_sequences(news_val)
 
padded_latih = pad_sequences(sekuens_latih,
                             padding='post',
                             maxlen=100,
                             truncating='post') 
padded_val = pad_sequences(sekuens_val,
                           padding='post',
                           maxlen=100,
                           truncating='post')

In [30]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=8000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', restore_best_weights=True, patience=20)

In [32]:
num_epochs = 50
batch_size = 128
history = model.fit(padded_latih, label_latih, epochs=num_epochs, batch_size = batch_size,
                    callbacks = [callback], validation_data=(padded_val, label_val), verbose=1)

Epoch 1/50
798/798 [==============================] - 97s 119ms/step - loss: 1.3855 - accuracy: 0.2488 - val_loss: 1.3839 - val_accuracy: 0.2558
Epoch 2/50
798/798 [==============================] - 93s 117ms/step - loss: 1.3869 - accuracy: 0.2528 - val_loss: 1.3860 - val_accuracy: 0.2530
Epoch 3/50
798/798 [==============================] - 94s 118ms/step - loss: 1.3848 - accuracy: 0.2518 - val_loss: 1.3839 - val_accuracy: 0.2550
Epoch 4/50
798/798 [==============================] - 96s 120ms/step - loss: 1.3025 - accuracy: 0.3229 - val_loss: 1.0891 - val_accuracy: 0.4518
Epoch 5/50
798/798 [==============================] - 94s 118ms/step - loss: 1.1125 - accuracy: 0.4390 - val_loss: 1.1222 - val_accuracy: 0.4374
Epoch 6/50
798/798 [==============================] - 95s 119ms/step - loss: 1.0995 - accuracy: 0.4653 - val_loss: 1.1337 - val_accuracy: 0.4397
Epoch 7/50
798/798 [==============================] - 96s 120ms/step - loss: 1.1446 - accuracy: 0.4346 - val_loss: 1.1676 - val_ac